In [1]:
!pip install census
!pip install us
!pip install addfips
!pip install requests

^C


In [26]:
#from importlib import metadata
from census import Census
from us import states
import API_KEYS
import pandas as pd
import requests
import numpy as np


c = Census(API_KEYS.API_KEY)

In [71]:
#CAREFUL THIS WILL GENERATE A LOT OF CSVs
'''
for i in range(2009, 2023):
    if i == 2009:
        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0064E', 'DP03_0063E', 'DP04_0003E', 'DP04_0005E', 'DP03_0103E', 'DP04_0088E','NAME'), '*', '*', year = i) 
        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value
        #https://api.census.gov/data/2009/acs/acs5/profile/variables/DP03_0006E.html
    if i >= 2015:
        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0063E', 'DP03_0062E', 'DP04_0003E', 'DP04_0005E', 'DP03_0119PE', 'DP04_0089E','NAME'), '*', '*', year = i)
        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value
        #https://api.census.gov/data/2022/acs/acs5/profile/variables.html
    else:
        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0063E', 'DP03_0062E', 'DP04_0003E', 'DP04_0005E', 'DP03_0119PE', 'DP04_0088E','NAME'), '*', '*', year = i) 
        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value
        #https://api.census.gov/data/2010/acs/acs5/profile/variables.html
    
    cen_data = pd.DataFrame(data)
    cen_data[['county_name', 'state_name']] = cen_data['NAME'].str.split(',', expand = True)
    cen_data.columns = ['mil pop', 'unemployment %', 'mean income', 'median income', 'vacant housing units', 'rental vacany rate', '% below poverty line', 'median house value', *cen_data.columns[8:]]
    cen_data.to_csv(str(i)+'_census_data.csv', index = False)
    print(i)
'''

"\nfor i in range(2009, 2023):\n    if i == 2009:\n        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0064E', 'DP03_0063E', 'DP04_0003E', 'DP04_0005E', 'DP03_0103E', 'DP04_0088E','NAME'), '*', '*', year = i) \n        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value\n        #https://api.census.gov/data/2009/acs/acs5/profile/variables/DP03_0006E.html\n    if i >= 2015:\n        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0063E', 'DP03_0062E', 'DP04_0003E', 'DP04_0005E', 'DP03_0119PE', 'DP04_0089E','NAME'), '*', '*', year = i)\n        #mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value\n        #https://api.census.gov/data/2022/acs/acs5/profile/variables.html\n    else:\n        data = c.acs5dp.state_county(('DP03_0006E', 'DP03_0009PE', 'DP03_0063E', 'DP03_0062E', 'DP04_0003E', 'DP04

In [4]:
#https://api.census.gov/data/1992/cps/asec/mar/variables.html
#P_STAT(2), 
#mil pop, unemployment %, mean income, median income, vacant housing units, rental vacany rate, % below poverty, median house value

In [59]:
#https://api.census.gov/data/1994/cps/basic/jan/variables.html
#PEAFNOW, PREXPLF (2/(1+2)), HUFAMINC (map to income levels), 
#mil pop, unemployment %, mean income, 
import requests
import API_KEYS

state_l = range(1,51)
state_l = str(list(state_l))[1:-1].replace(' ','')

response = requests.get('http://api.census.gov/data/1994/cps/basic/jan?get=PEAFNOW,PREXPLF,HUFAMINC&for=county:*&in=state:'+state_l+'&key='+API_KEYS.API_KEY)
response2 = requests.get('http://api.census.gov/data/1994/cps/basic/feb?get=PEAFNOW,PREXPLF,HUFAMINC&for=county:*&in=state:'+state_l+'&key='+API_KEYS.API_KEY)

In [6]:
import pandas as pd
import json


response.raise_for_status()  # raises exception when not a 2xx response
if response.status_code != 204:
    response.json()

In [50]:
fips_df = pd.read_csv('county_fips_master.csv', encoding='ISO-8859-1')
fips_df['fips'] = fips_df['fips'].astype(str).str.zfill(5)

In [72]:
#CAREFUL THIS WILL GENERATE A LOT OF CSVs
'''
month_list = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]

for y in range(1994, 2009):
    df_list = []
    response_list = []
    for m in month_list:
        url = 'http://api.census.gov/data/'+str(y)+'/cps/basic/'+m+'?get=PEAFNOW,PREXPLF,HUFAMINC&for=county:*&in=state:'+state_l+'&key='+API_KEYS.API_KEY
        response = requests.get(url)
        print(m, y)
        response_list.append(response)
        df=pd.DataFrame(response.json())
        df.columns = df.loc[0]
        df = df.loc[1::]
        df_list.append(df)

    df = pd.concat(df_list)


    df['county_fips'] = df['county'].str.zfill(3)
    df['state_fips'] = df['state'].str.zfill(2)

    df['fips'] = df['state_fips'] + df['county_fips']

    df = df.merge(fips_df, left_on='fips', right_on='fips')

    df['PEAFNOW'] = df['PEAFNOW'].astype(int)

    num = df[df['PEAFNOW'] == 1].groupby(by='fips')['PEAFNOW'].sum()
    den = df.groupby(by='fips')['PEAFNOW'].sum()

    ratio_df = pd.DataFrame(num/den)

    df = df.merge(ratio_df, on = 'fips')
    df.rename(columns = {'PEAFNOW_y': '% active duty'}, inplace = True)

    income_map = {
          "-1": np.nan,
          "7": (20000 + 24999)/2,
          "-2": np.nan,
          "10": (35000 + 39999)/2,
          "5": (12500 + 14999)/2,
          "1": 5000,
          "12": (50000 + 59999)/2,
          "3": (7500 + 9999)/2,
          "4": (10000 + 12499)/2,
          "11": (40000 + 49999)/2,
          "13": (60000 + 74999)/2,
          "14": 75000,
          "8": (25000 + 29999)/2,
          "2": (5000 + 7499)/2,
          "-3": np.nan,
          "9": (30000 + 34999)/2,
          "6": (15000 + 19999)/2
        }

    df['Yearly Income'] = df['HUFAMINC'].map(income_map)
    df[df['% active duty'].notna()]

    num = df[df['PREXPLF'] == '2'].groupby(by='fips')['PREXPLF'].count()
    den = df[df['PREXPLF'].isin(['1','2'])].groupby(by='fips')['PREXPLF'].count()

    ratio_df = pd.DataFrame(num/den)

    df = df.merge(ratio_df, on = 'fips')
    df.rename(columns = {'PREXPLF_y': '% unemployed'}, inplace = True)
    out = df.groupby(by = 'fips').mean()[['% active duty', '% unemployed', 'Yearly Income']]
    out.to_csv(m+'_'+str(y)+'.csv')
'''

jan 2007
feb 2007
mar 2007
apr 2007
may 2007
jun 2007
jul 2007
aug 2007
sep 2007
oct 2007
nov 2007
dec 2007
jan 2008
feb 2008
mar 2008
apr 2008
may 2008
jun 2008
jul 2008
aug 2008
sep 2008
oct 2008
nov 2008
dec 2008


In [19]:
#median house value
df_house = pd.read_excel('hpi_at_bdl_county.xlsx', skiprows = 6) #https://www.fhfa.gov/data/hpi/datasets?tab=additional-data

In [20]:
df_house

,State,County,FIPS code,Year,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,AL,Autauga,1001,1986,.,100.00,95.59,70.96
1,AL,Autauga,1001,1987,-1.94,98.06,93.74,69.59
2,AL,Autauga,1001,1988,2.57,100.58,96.14,71.38
3,AL,Autauga,1001,1989,4.32,104.92,100.29,74.46
4,AL,Autauga,1001,1990,-0.29,104.62,100.00,74.24
...,...,...,...,...,...,...,...,...
100427,WY,Weston,56045,2019,8.33,223.40,.,201.09
100428,WY,Weston,56045,2020,4.36,233.13,.,209.86
100429,WY,Weston,56045,2021,4.93,244.62,.,220.20
100430,WY,Weston,56045,2022,6.87,261.41,.,235.32


In [ ]:
#TO FIND PRE 2009
#median income, vacant housing units, rental vacany rate, % below poverty, 